In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 15.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=a6de20783f10bba52fa78e1847b07bc1b862b1720f8984a4eb0e8c52f11120fe
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StandardScaler, Word2Vec, StopWordsRemover, VectorAssembler, StringIndexer, HashingTF, IDF, Tokenizer
import pyspark.sql.functions as f
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import pandas as pd


spark = SparkSession \
    .builder \
    .appName("HTFIDF") \
    .master("local[*]") \
    .config("spark.executor.memory", "70g") \
    .config("spark.driver.memory", "50g") \
    .config("spark.memory.offHeap.enabled",True) \
    .config("spark.memory.offHeap.size","16g") \
    .getOrCreate()

sc=spark.sparkContext

df = spark.read.option("header",True).option("quote", "\"").option("escape", "\"").option("multiline", True).csv("../input/mlbd-hw05/train.csv")
df.printSchema()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/09 15:51:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
root
 |-- id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- toxic: string (nullable = true)
 |-- severe_toxic: string (nullable = true)
 |-- obscene: string (nullable = true)
 |-- threat: string (nullable = true)
 |-- insult: string (nullable = true)
 |-- identity_hate: string (nullable = true)



In [4]:
#пробный подход

In [8]:
new_df = df.withColumn('comment_text', f.lower(f.col('comment_text')))
#Токенизуем и удалим стоп слова
tokenizer = Tokenizer(inputCol='comment_text', outputCol='words')
new_df = tokenizer.transform(new_df)
remover = StopWordsRemover(inputCol='words', outputCol='filtered')
new_df = remover.transform(new_df)

In [12]:
#пайплайн
NUM_FEATURES = 10

hashing = HashingTF(inputCol='filtered', outputCol='raw_features', numFeatures=NUM_FEATURES)
idf = IDF(inputCol='raw_features', outputCol='features')
label_stringIdx = StringIndexer(inputCol = "toxic", outputCol = "label")
lr = LogisticRegression(maxIter=10)

pipeline = Pipeline(stages= [hashing, idf, label_stringIdx, lr])
model = pipeline.fit(new_df)

rescale_df = model.transform(new_df)
rescale_df.select('label', 'prediction').show(truncate=False)

+-----+----------+
|label|prediction|
+-----+----------+
|0.0  |0.0       |
|0.0  |0.0       |
|0.0  |0.0       |
|0.0  |0.0       |
|0.0  |0.0       |
|0.0  |0.0       |
|1.0  |0.0       |
|0.0  |0.0       |
|0.0  |0.0       |
|0.0  |0.0       |
|0.0  |0.0       |
|0.0  |0.0       |
|1.0  |0.0       |
|0.0  |0.0       |
|0.0  |0.0       |
|0.0  |0.0       |
|1.0  |0.0       |
|0.0  |0.0       |
|0.0  |0.0       |
|0.0  |0.0       |
+-----+----------+
only showing top 20 rows



In [20]:
param_grid = ParamGridBuilder().addGrid(
    hashing.numFeatures, [10, 20, 50, 100, 200]
).build()

#roc-auc - окей :)
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid, evaluator=BinaryClassificationEvaluator(), numFolds=5)

cvModel = cv.fit(new_df)

22/12/08 17:04:05 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/12/08 17:04:05 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [21]:
params = [{p.name: v for p, v in m.items()} for m in cvModel.getEstimatorParamMaps()]

pd.DataFrame.from_dict([
    {cvModel.getEvaluator().getMetricName(): metric, **ps} 
    for ps, metric in zip(params, cvModel.avgMetrics)
])

,areaUnderROC,numFeatures
0,0.637856,10
1,0.654037,20
2,0.702389,50
3,0.739625,100
4,0.829281,1000


In [ ]:
#2

In [25]:
def tfidfcv(target_name, df = df, num_feature_list = [10, 20, 50, 100, 200]):
    new_df = df.withColumn('comment_text', f.lower(f.col('comment_text')))
    tokenizer = Tokenizer(inputCol='comment_text', outputCol='words')
    new_df = tokenizer.transform(new_df)
    remover = StopWordsRemover(inputCol='words', outputCol='filtered')
    new_df = remover.transform(new_df)
    
    hashing = HashingTF(inputCol='filtered', outputCol='raw_features', numFeatures=10)
    idf = IDF(inputCol='raw_features', outputCol='features')
    label_stringIdx = StringIndexer(inputCol = target_name, outputCol = "label")
    lr = LogisticRegression(maxIter=10)

    pipeline = Pipeline(stages= [hashing, idf, label_stringIdx, lr])
    param_grid = ParamGridBuilder().addGrid(
    hashing.numFeatures, [10, 20, 50, 100, 200]
    ).build()
    cv = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid, evaluator=BinaryClassificationEvaluator(), numFolds=5)
    cvModel = cv.fit(new_df)
    params = [{p.name: v for p, v in m.items()} for m in cvModel.getEstimatorParamMaps()]
    return pd.DataFrame.from_dict([
        {cvModel.getEvaluator().getMetricName(): metric, **ps} 
        for ps, metric in zip(params, cvModel.avgMetrics)
    ])

toxic_results = tfidfcv('toxic')
obscene_results = tfidfcv('obscene')
threat_results = tfidfcv('threat')
insult_results = tfidfcv('insult')
identity_hate_results = tfidfcv('identity_hate')

22/12/08 17:16:51 WARN MemoryStore: Not enough space to cache rdd_5174_0 in memory! (computed 257.4 MiB so far)
22/12/08 17:16:51 WARN BlockManager: Persisting block rdd_5174_0 to disk instead.


22/12/08 17:16:55 WARN MemoryStore: Not enough space to cache rdd_5174_0 in memory! (computed 130.7 MiB so far)


22/12/08 17:17:00 WARN MemoryStore: Not enough space to cache rdd_5174_0 in memory! (computed 130.7 MiB so far)


22/12/08 17:17:01 WARN MemoryStore: Not enough space to cache rdd_5174_0 in memory! (computed 130.7 MiB so far)


22/12/08 17:17:07 WARN MemoryStore: Not enough space to cache rdd_5174_0 in memory! (computed 130.7 MiB so far)


...

22/12/08 17:18:36 WARN MemoryStore: Not enough space to cache rdd_5174_0 in memory! (computed 257.4 MiB so far)


22/12/08 17:18:45 WARN MemoryStore: Not enough space to cache rdd_5174_0 in memory! (computed 257.4 MiB so far)


22/12/08 17:18:46 WARN MemoryStore: Not enough space to cache rdd_5133_0 in memory! (computed 257.4 MiB so far)
22/12/08 17:18:51 WARN MemoryStore: Not enough space to cache rdd_5800_0 in memory! (computed 17.0 MiB so far)
22/12/08 17:18:51 WARN BlockManager: Persisting block rdd_5800_0 to disk instead.


22/12/08 17:19:02 WARN MemoryStore: Not enough space to cache rdd_5174_0 in memory! (computed 257.4 MiB so far)


22/12/08 17:59:53 WARN BlockManager: Asked to remove block broadcast_6519, which does not exist


In [27]:
toxic_results['target'] = "toxic"
obscene_results['target'] = "obscene"
threat_results['target'] = "threat"
insult_results['target'] = "insult"
identity_hate_results['target'] = "identity_hate"
pd.concat([toxic_results, obscene_results, threat_results, identity_hate_results])

,areaUnderROC,numFeatures,target
0,0.637856,10,toxic
1,0.654037,20,toxic
2,0.702389,50,toxic
3,0.739625,100,toxic
4,0.772005,200,toxic
0,0.657923,10,obscene
1,0.669138,20,obscene
2,0.748032,50,obscene
3,0.783319,100,obscene
4,0.811158,200,obscene


In [10]:
#w2v

In [3]:
def w2vcv(target_name, df = df, num_feature_list = [10, 20, 50, 100, 200]):
    new_df = df.withColumn('comment_text', f.lower(f.col('comment_text')))
    tokenizer = Tokenizer(inputCol='comment_text', outputCol='words')
    new_df = tokenizer.transform(new_df)
    remover = StopWordsRemover(inputCol='words', outputCol='filtered')
    new_df = remover.transform(new_df)
    VECTORSIZE = 3
    word2Vec = Word2Vec(vectorSize=VECTORSIZE, minCount=0, inputCol='filtered', outputCol='features')
    label_stringIdx = StringIndexer(inputCol = target_name, outputCol = "label")
    lr = LogisticRegression(maxIter=10)

    pipeline = Pipeline(stages= [word2Vec, label_stringIdx, lr])
    param_grid = ParamGridBuilder().addGrid(
    word2Vec.vectorSize, [3]
    ).build()
    cv = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid, evaluator=BinaryClassificationEvaluator(), numFolds=5)
    cvModel = cv.fit(new_df)
    params = [{p.name: v for p, v in m.items()} for m in cvModel.getEstimatorParamMaps()]
    return pd.DataFrame.from_dict([
        {cvModel.getEvaluator().getMetricName(): metric, **ps} 
        for ps, metric in zip(params, cvModel.avgMetrics)
    ])

toxic_w2v_results = w2vcv('toxic')
obscene_w2v_results = w2vcv('obscene')
threat_w2v_results = w2vcv('threat')
insult_w2v_results = w2vcv('insult')
identity_w2v_hate_results = w2vcv('identity_hate')

In [4]:
toxic_w2v_results['target'] = "toxic"
obscene_w2v_results['target'] = "obscene"
threat_w2v_results['target'] = "threat"
insult_w2v_results['target'] = "insult"
identity_w2v_hate_results['target'] = "identity_hate"
pd.concat([toxic_w2v_results, obscene_w2v_results, threat_w2v_results, insult_w2v_results, identity_w2v_hate_results])

,areaUnderROC,vectorSize,target
0,0.793609,3,toxic
0,0.863533,3,obscene
0,0.854100,3,threat
0,0.845838,3,insult
0,0.889761,3,identity_hate
